# VQE with LabOne Q and Qiskit

The following example demonstrates a simple end-to-end implementation of the variational quantum eigensolver (VQE) with LabOne Q.

Qiskit is used to generate the quantum circuits to prepare a parameterized state ansatz and to perform the transformations needed to compute the Hamiltonian expectation value.

The circuits are then exported as OpenQASM programs and subsequently transformed into a LabOne Q experiment.
The compiled experiment is then used in an example implementation of a VQE objective function that updates the parameters of the state ansatz, executes the experiment, and returns the expectation value.
Finally, this objective function is then minimized with the help of an external optimizer.

Note, there are many ways to improve the efficiency of the VQE algorithm.
The comparatively simple implementation demonstrated in this example serves to illustrate basic concepts and might serve as a starting point for users to optimize its performance.


## Imports

In [116]:
# Numpy:
import numpy as np

# scipy optimizer
from scipy.optimize import minimize

# Qiskit:
from qiskit import qasm3, QuantumCircuit
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# LabOne Q:
from laboneq.simple import *
from laboneq._utils import id_generator
from laboneq.contrib.example_helpers.generate_example_datastore import (
    generate_example_datastore,
    get_first_named_entry,
)
from laboneq.openqasm3.gate_store import GateStore

# Enable/disable emulation mode
use_emulation = True

## Hamiltonian (Qiskit)
The variational quantum eigensolver (VQE) is an algorithm to approximate the ground state $\psi_{0}$ with eigenvalue $\epsilon_{0}$ of a problem described by a Hamiltonian $\hat{H}$.
VQE achieves this by minimizing the expectation value of $\hat{H}$

$\epsilon_{\theta} = \langle\psi_{\theta}|\hat{H}|\psi_{\theta}\rangle$

with respect to a trial ansatz $\psi_{\theta}$ parameterized by $\theta$.
Because of the variational principle $\epsilon_{\theta}$ is always an upper bound for $\epsilon_{0}$.
A successful minimization of $\epsilon_{\theta}$ will thus provide an increasingly accurate approximation to the sought solution.

VQE requires the repeated computation of expectation values.
This can be done efficiently for Hamiltonians which have the form of a (sparse) series,

$\hat{H} = \sum\limits_{i}c_{i}\ \hat{P}_{i}$

of multi-qubit Pauli operators $\hat{P}_{i} \in \{\hat{I}, \hat{X}, \hat{Y}, \hat{Z}\}^{\otimes n}$ with coefficients $c_{i}$.

Quantum computing software packages like Qiskit have dedicated modules that construct Hamiltonians in this form for various quantum computational applications.
For this example, we use Qiskit's `SparsePauliOp` class to define such a Hamiltonian explicitly.


In [117]:
H = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -1.0113), ("XX", 0.1810)]
)

## Parameterized wavefunction ansatz (Qiskit)

In this section, we construct a parameterized circuit that prepares the trial state $\psi_{\theta}$.
Qiskit's `EfficientSU2` class allows us to define an ansatz for this state.
For two qubits this results in four layers.
Each layer applies two parameterized SU2 gates (here `Rx` and `Ry`) to each qubit, respectively, followed by a `cnot` gate to create an entangled state.

Note, that we use the `Rx`,`Ry` pair of SU2 gates over the default `Ry`, `Rz` pair since optimizing over virtual z-gate angles would require replacing phase increments in compiled LabOne Q experiments, which at the time of writing is still an experimental feature.

The resulting circuit is appended to the initial state reset.


In [ ]:
trial_state = EfficientSU2(H.num_qubits, su2_gates=["rx", "ry"], insert_barriers=True)
trial_state.barrier()

trial_state.decompose().draw(style="iqp")

The above trial state $|\psi_{\theta}\rangle$ is parameterized by $\theta=(\theta_{0}, \theta_{1}, \dots, \theta_{15})$.


## Transformation to Pauli eigenbasis (Qiskit)

After preparing the parameterized trial state $|\psi_{\theta}\rangle$ we need to evaluate the Hamiltonian expectation value for this state.
We note the following aspects:

* Expectation values are computed for each Pauli term $\hat{P}_{i}$ individually. For the sake of simplicity, we use circuits and ultimately separate LabOne Q experiments. We note here, that optimizing the approach discussed here is 
* We can only obtain information from a quantum circuit by measuring states in the computational basis $\left\{|0\rangle, |1\rangle\right\}$.
* The computational basis of a qubit corresponds to the eigenfunctions of the Pauli $\hat{Z}$ operator acting on that qubit. All occurrences of $\hat{X}$ or $\hat{Y}$ in a Pauli term are therefore transformed into $\hat{Z}$ operators by applying a transformation $\hat{U}$ to the trial state $|\psi_{\theta}\rangle$.

<!-- We obtain $\epsilon_{\theta}$ as expectation value of individual Pauli terms

$\langle\psi_{\theta}|\hat{P}|\psi_{\theta}\rangle = \langle\psi_{\theta}\hat{U}^{\dagger}|\hat{Z}\otimes\hat{1}|\hat{U}\psi_{\theta}\rangle$,

with the unitary operator $\hat{U}$ transforming $\hat{P}$ into $\hat{Z}\otimes\hat{1}$.
Recall, that $\hat{P}$ has eigenvalues of $\pm1$.
The corresponding eigenfunctions with eigenvalue +1 are thereby projected by $\hat{U}$ onto states corresponding to measuring the first qubit in $|0q_{1}q_{2}\cdots\rangle$.
Likewise, eigenfunctions of $\hat{P}$ with negative eigenvalues are projected into the subspace corresponding to measuring this qubit in $|1q_{1}q_{2}\cdots\rangle$.
Furthermore, 

 $\langle\psi_{\theta}|\hat{P}|\psi_{\theta}\rangle = |a|^{2}\langle0\cdots|H|0\cdots\rangle + |b|^{2}\langle1\cdots|H|1\cdots\rangle = |a|^{2} - |b|^{2}$,

directly relates the expectation value to the coefficients $a$ and $b$, which are determined by a state tomography process.

Here, we define the required transformations for each Pauli term separately. -->


In [119]:
vqe_terms = []
for p in H.paulis:
    P_i = p.to_label()
    u_transform = QuantumCircuit(H.num_qubits, H.num_qubits)

    for q in range(H.num_qubits):
        if P_i[q] == "X":
            u_transform.h(q)

        elif P_i[q] == "Y":
            u_transform.sdg(q)
            u_transform.h(q)

    # u_transform.barrier()

    # for q in range(H.num_qubits):
    #     u_transform.measure(q, q)

    vqe_terms.append(trial_state & u_transform)

We can now inspect the final circuits of the VQE algorithm.

In [ ]:
vqe_terms[0].decompose().draw(style="iqp")
# vqe_terms[1].decompose().draw(style="iqp")
# vqe_terms[2].decompose().draw(style="iqp")
# vqe_terms[3].decompose().draw(style="iqp")

## Conversion to OpenQASM code (Qiskit)
We can use Qiskit's qasm3 functionality to turn the circuits defined above into qasm programs. 


In [ ]:
qasm_programs = [qasm3.dumps(c.decompose()) for c in vqe_terms]

print(qasm_programs[0])

Note, that the parameters now appear as input variables `_θ_0_` to `_θ_15_`.

## Parameter Initialization

When later assigning values to these parameters, we will need to provide matching parameter names.

Also parameter updates to the objective function are achieved by replacing the pulses containing these parameters.
For this we need to be able to identify such parameter dependent pulses in the compiled experiments.

To achieve this, we define the `ScalarParameter` class that holds the parameter value as well as its name.

In [122]:
class ScalarParameter(object):
    def __init__(self, val, id):
        self.val = val
        self.id = id

We initialize the parameters with random values in the interval $[0, 1)$.

In [123]:
np.random.seed(seed=42)
theta = [
    ScalarParameter(np.random.random(), f"_θ_{i}_")
    for i in range(trial_state.num_parameters)
]

## Gate Definitions (LabOne Q)
The construction of circuits and qasm programs made use of abstract gate definitions.
To run the VQE algorithm on actual hardware, we need to define the gates $U_1$, $U_2$, $CX$, $R_x$, and $R_y$ as well as reset and measurement operations in terms of pulse sequences in LabOne Q.

Note that the definitions below are for demonstration purposes. Actual gate and pulse implementations are typically hardware-specific might even vary for individual qubits.

In [124]:
def u1(qubit: Qubit):
    """Return a parameterized U1 gate for the specified qubit.
    The gate is a function that takes the angle to rotate and
    returns a LabOne Q section that performs the rotation.
    """

    def u1_gate(angle):
        """equivalent to Rz(theta) as U1(theta) differs from it only by a global phase which is ignored.
        Theta is in radians - implements a virtual z-gate
        """
        gate = Section(uid=id_generator(f"p_{qubit.uid}_u1_{int(180 * angle / np.pi)}"))
        gate.play(
            signal=qubit.signals["drive"],
            pulse=None,
            increment_oscillator_phase=angle,
        )
        return gate

    return u1_gate

In [125]:
def u2(qubit: Qubit):
    """Return a parameterized U2 gate for the specified qubit.
    The gate is a function that takes two angles to rotate and
    returns a LabOne Q section that performs the rotation.
    """

    def u2_gate(phi, lam):
        """U2(ϕ,λ)=RZ(ϕ).RY(π/2).RZ(λ) - following definition in qiskit.circuit.library.U2Gate
        phi and lambda are in radians and serve to implement the two virtual z-gates
        """
        pulse = pulse_library.drag(
            uid=f"{qubit.uid}_ry_in_u2",
            length=qubit.parameters.user_defined["pulse_length"],
            amplitude=qubit.parameters.user_defined["amplitude_pi"] / 2,
        )
        gate = Section(
            uid=id_generator(
                f"p_{qubit.uid}_u2_{int(180 * phi / np.pi)}_{int(180 * lam / np.pi)}"
            )
        )
        gate.play(
            signal=qubit.signals["drive"],
            pulse=None,
            increment_oscillator_phase=lam,
        )
        gate.play(
            signal=qubit.signals["drive"],
            pulse=pulse,
            phase=np.pi / 2,
        )
        gate.play(
            signal=qubit.signals["drive"],
            pulse=None,
            increment_oscillator_phase=phi,
        )
        return gate

    return u2_gate

In [126]:
def cx(control: Qubit, target: Qubit):
    """Return a controlled X gate for the specified control and target qubits.

    The CX gate function takes no arguments and returns a LabOne Q section that performs
    the controlled X gate.
    """

    def cx_gate():
        cx_id = f"cx_{control.uid}_{target.uid}"

        gate = Section(uid=id_generator(cx_id))

        # define X pulses for target and control
        x180_pulse_control = pulse_library.drag(
            uid=f"{control.uid}_x180",
            length=control.parameters.user_defined["pulse_length"],
            amplitude=control.parameters.user_defined["amplitude_pi"],
        )
        x180_pulse_target = pulse_library.drag(
            uid=f"{target.uid}_x180",
            length=target.parameters.user_defined["pulse_length"],
            amplitude=target.parameters.user_defined["amplitude_pi"],
        )

        # define cancellation pulses for target and control
        cancellation_control_n = pulse_library.gaussian_square(uid="CR-")
        cancellation_control_p = pulse_library.gaussian_square(uid="CR+")
        cancellation_target_p = pulse_library.gaussian_square(uid="q1+")
        cancellation_target_n = pulse_library.gaussian_square(uid="q1-")

        # play X pulses on both target and control
        x180_both = Section(uid=id_generator(f"{cx_id}_x_both"))
        x180_both.play(signal=control.signals["drive"], pulse=x180_pulse_control)
        x180_both.play(signal=target.signals["drive"], pulse=x180_pulse_target)
        gate.add(x180_both)

        # First cross-resonance component
        cancellation_p = Section(
            uid=id_generator(f"{cx_id}_canc_p"), play_after=x180_both
        )
        cancellation_p.play(signal=target.signals["drive"], pulse=cancellation_target_p)
        cancellation_p.play(
            signal=control.signals["flux"], pulse=cancellation_control_n
        )
        gate.add(cancellation_p)

        # play X pulse on control
        x180_control = Section(
            uid=id_generator(f"{cx_id}_x_q0"), play_after=cancellation_p
        )
        x180_control.play(signal=control.signals["drive"], pulse=x180_pulse_control)
        gate.add(x180_control)

        # Second cross-resonance component
        cancellation_n = Section(
            uid=id_generator(f"cx_{cx_id}_canc_n"), play_after=x180_control
        )
        cancellation_n.play(signal=target.signals["drive"], pulse=cancellation_target_n)
        cancellation_n.play(
            signal=control.signals["flux"], pulse=cancellation_control_p
        )
        gate.add(cancellation_n)

        return gate

    return cx_gate

In [127]:
def measurement(qubit: Qubit):
    """Return a measurement operation of the specified qubit.

    The operation is a function that takes the measurement handle (a string)
    and returns a LabOne Q section that performs the measurement.
    """

    def measurement_gate(handle: str):
        """Perform a measurement.

        Handle is the name of where to store the measurement result. E.g. "meas[0]".
        """
        measure_pulse = pulse_library.gaussian_square(
            uid=f"{qubit.uid}_readout_pulse",
            length=qubit.parameters.user_defined["readout_length"],
            amplitude=qubit.parameters.user_defined["readout_amplitude"],
        )
        integration_kernel = pulse_library.const(
            uid=f"{qubit.uid}_integration_kernel",
            length=qubit.parameters.user_defined["readout_length"],
        )
        gate = Section(uid=id_generator(f"meas_{qubit.uid}_{handle}"))
        gate.reserve(signal=qubit.signals["drive"])
        gate.play(signal=qubit.signals["measure"], pulse=measure_pulse)
        gate.acquire(
            signal=qubit.signals["acquire"],
            handle=handle,
            kernel=integration_kernel,
        )
        return gate

    return measurement_gate

In [128]:
def reset(qubit: Qubit):
    """Reset the specified qubit to the ground state with the supplied reset pulse.

    The reset gate function takes no arguments and returns a LabOne Q section that performs
    the reset.
    """

    def reset_gate():
        reset_pulse = pulse_library.drag(
            uid=f"{qubit.uid}_reset",
            length=qubit.parameters.user_defined["pulse_length"],
            amplitude=qubit.parameters.user_defined["amplitude_pi"],
        )
        # Reset Section
        reset = Section(uid=f"{qubit.uid}_reset")
        # qubit state readout
        readout = measurement(qubit)(f"{qubit.uid}_reset")
        # delay after measurement
        readout.delay(
            signal=qubit.signals["acquire"],
            time=qubit.parameters.user_defined["reset_delay_length"],
        )
        # real-time feedback, fetching the measurement data identified by handle locally from the QA unit of the SHFQC
        match_case = Match(
            uid=f"{qubit.uid}_feedback",
            handle=f"{qubit.uid}_reset",
            play_after=readout,
        )
        # measurement result 0 - ground state
        case_0 = Case(uid=f"{qubit.uid}_0_Case", state=0)
        case_0.play(signal=qubit.signals["drive"], pulse=reset_pulse, amplitude=0.01)
        # measurement result 1 - excited state
        case_1 = Case(uid=f"{qubit.uid}_1_Case", state=1)
        # play x180 pulse
        case_1.play(signal=qubit.signals["drive"], pulse=reset_pulse)
        match_case.add(case_0)
        match_case.add(case_1)

        reset.add(readout)
        reset.add(match_case)
        return reset

    return reset_gate

The remaining `Rx` and `Ry` gates depend on the parameters θ.

The following function generates a drag pulse from an amplitude parameter. 

In [129]:
def pulse_from_parameter(theta):
    val = theta.val % 2.0  # full rotations
    id = f"pulse{theta.id}"
    return pulse_library.drag(uid=id, amplitude=val if val <= 1.0 else val - 2.0)

In [130]:
def rx(qubit: Qubit):
    """Return a parameterized Rx gate for the specified qubit.
    The gate is a function that takes the angle to rotate and
    returns a LabOne Q section that performs the rotation.
    """

    def rx_gate(angle: ScalarParameter):
        """Rx(theta).
        Theta is in units of pi - pulse amplitude is adjusted according to the chosen angle
        """
        gate = Section(uid=id_generator(f"p_{qubit.uid}_rx_{int(180 * angle.val)}"))
        gate.play(
            signal=qubit.signals["drive"],
            pulse=pulse_from_parameter(angle),
            amplitude=qubit.parameters.user_defined["amplitude_pi"],
        )
        return gate

    return rx_gate

In [131]:
def ry(qubit: Qubit):
    """Return a parameterized Ry gate for the specified qubit.

    The gate is a function that takes the angle to rotate and
    returns a LabOne Q section that performs the rotation.
    """

    def ry_gate(angle: ScalarParameter):
        """Ry(theta).

        Theta is in units of pi - pulse amplitude is adjusted according to the chosen angle
        """
        gate = Section(uid=id_generator(f"p_{qubit.uid}_ry_{int(180 * angle.val)}"))

        gate.play(
            signal=qubit.signals["drive"],
            pulse=pulse_from_parameter(angle),
            amplitude=qubit.parameters.user_defined["amplitude_pi"],
            phase=np.pi / 2,
        )
        return gate

    return ry_gate

## Device setup, qubits, and gate store (LabOne Q)

Before we can proceed with LabOne Q, we need to define the necessary qubit objects (and their respective parameters). Here we use a predefined `device_setup` instance from a database.


In [132]:
# load a calibrated device setup from the dummy database
device_setup = get_first_named_entry(
    db=generate_example_datastore(path="", filename=":memory:"),
    name="6_tuneable_qubit_setup_shfqc_hdawg_pqsc_calibrated",
)

The `device_setup` contains the qubit objects whose parameters will be used during the gate operations.
In this example only the first two qubits are needed.

In [133]:
q0, q1 = device_setup.qubits[:2]

The gate store  defines the mapping between logical operations (i.e. those that appear in OpenQASM statements) and the physical operations (i.e. functions that define LabOne Q sections to play) above.

The qubit names provided in this mapping therefore need to match the qubit names of the OpenQASM program.


In [134]:
# Initialize
gate_store = GateStore()

# map LabOne Q qubits to qasm qubits
qubit_map = {"q[0]": device_setup.qubits[0], "q[1]": device_setup.qubits[1]}

# Single qubit gates:
for q in qubit_map:
    gate_store.register_gate_section("rx", (q,), rx(qubit_map[q]))
    gate_store.register_gate_section("ry", (q,), ry(qubit_map[q]))
    gate_store.register_gate_section("u1", (q,), u1(qubit_map[q]))
    gate_store.register_gate_section("u2", (q,), u2(qubit_map[q]))
    gate_store.register_gate_section("measure", (q,), measurement(qubit_map[q]))
    gate_store.register_gate_section("reset", (q,), reset(qubit_map[q]))

# Two qubit gates:
gate_store.register_gate_section("cx", ("q[0]", "q[1]"), cx(q0, q1))

## LabOne Q Experiment (LabOne Q)

We are now ready to use LabOne Q's `exp_from_qasm_list` function to convert the OpenQASM programs into a single `Experiment` instance and note the following

* the initial parameter values are assigned to the `inputs` argument
* `averaging_mode` and `acquisition_type` are set to allow to use the state indices for the computation of the expectation value
* we set the repetition time to the approximative length of the VQE sequence manually
* the `do_reset` option is used to initialize all qubits to the ground state before each VQE sequence
* the `add_measurement` option allows to add measurements of all qubit after each VQE sequence

In [135]:
exp = exp_from_qasm_list(
    qasm_programs,
    qubits=qubit_map,
    gate_store=gate_store,
    inputs={t.id: t for t in theta},
    count=1024,
    averaging_mode=AveragingMode.SINGLE_SHOT,
    acquisition_type=AcquisitionType.DISCRIMINATION,
    batch_execution_mode="pipeliner",
    repetition_time=2.2e-6,
    do_reset=True,
    add_measurement=True,
)

A `Session` instance is created and used to compile all experiments.

In [ ]:
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

cexp = session.compile(exp)

We can use the pulse sheet viewer to inspect the compiled experiment.
Note, that the VQE experiments are relatively long and we need to raise the number of events to see a full sequence of all VQE terms.

In [ ]:
show_pulse_sheet("vqe", cexp, max_events_to_publish=3000)

## VQE objective function

We have now established the parts needed to compute the expectation value $\epsilon_{\theta} = \langle\psi_{\theta}|\hat{H}|\psi_{\theta}\rangle$.
For this we first define the following auxiliary functions:

At present, the emulation mode does not provide the the qubit state indices. To make the acquired results compatible, the function `format_emulated_results` defines the measured qubit states as random integer vectors with entries $\{0, 1\}$.

In [138]:
def format_emulated_results(results, num_qubits):
    for i in range(num_qubits):
        results.acquired_results[f"measq[{i}]"].data = np.array(
            np.random.randint(
                0, 2, size=results.acquired_results[f"measq[{i}]"].data.shape
            )
        )
    return results

The function `expectation_value_VQE` computes $\langle\psi_{\theta}|\hat{P}_{i}|\psi_{\theta}\rangle$.
* The acquired results (qubit state indices) are collected into a single array with axes spanning 1. qubits, 2. single shots, and 3. VQE terms.
* The expectation value of each multi-qubit Pauli operator is either $1$ or $-1$ depending on whether an even or odd number of contributing qubits is found in state 1. Note, that this corresponds to the parity of the sum of the state indices. The parity is computed over all qubits that contribute to the expectation value, i.e. onto which a Pauli operator acts.
* The resulting parities are then averaged over the number of single shots, converted into expectation values contributions and summed up to the final $\epsilon_{\theta}$.

In [139]:
def expectation_value_VQE(acquired_results, H):
    # sort measurement data
    state_indices = np.array(
        [acquired_results[f"measq[{q}]"].data for q in range(H.num_qubits)]
    ).transpose((2, 1, 0))
    # expectation value
    exp_val = 0.0
    for i, (P, c) in enumerate(H.to_list()):
        exp_val += (
            c.real
            - c.real
            * np.mean(
                np.sum(state_indices[i], axis=-1, where=[p for p in P if p != "I"]) % 2,
                axis=-1,
            )
            * 2.0
        )
    return exp_val

Finally, we collect all the above functionality for parameter updates, experiment execution, results emulation, and VQE-postprocessing into a single objective function that returns $\langle\psi_{\theta}|\hat{P}_{i}|\psi_{\theta}\rangle$ in terms of a vector of parameter values. 

In [140]:
def objective_function_VQE(H, theta, cexp, session):
    def hamiltonian_expectation_value(x):
        if use_emulation:  # ensure consistent results at every pass
            np.random.seed(seed=10101)

        # Parameter Update
        for xx, t in zip(x, theta):
            t.val = xx
            new_pulse = pulse_from_parameter(t)
            cexp.replace_pulse(new_pulse.uid, new_pulse)

        # Execute Experiment
        res = session.run(cexp)

        # Format Results
        if use_emulation:
            res = format_emulated_results(res, H.num_qubits)

        # Obtain expectation value
        y = expectation_value_VQE(res.acquired_results, H)

        # Log iteration result and return
        print(f"VQE expectation value {y:20.4e}")
        return y

    return hamiltonian_expectation_value

## Optimization

The objective function constructed in the previous section can now directly be used for minimization.
We use the "constrained optimization by linear approximation" (COBYLA) method from the scipy library for this.
However, any gradient-less optimization method should be suitable for this step. 

Note, that we restrict the number of evaluation steps to 1 in emulation mode as we would otherwise optimize a constant function.

In [ ]:
res = minimize(
    fun=objective_function_VQE(H, theta, cexp, session),
    x0=[t.val for t in theta],
    method="COBYLA",
    options={"maxiter": 1} if use_emulation else {},
)

Finally, we can retrieve $\min\limits_{\theta}\{\epsilon_{\theta}\}$ as VQE approximation of the ground state energy of $\hat{H}$

In [ ]:
res.fun

We can also inspect the full results object returned by the minimizer for the metainformation of the optimization.

In [ ]:
res